In [32]:
# Import packages

import pandas as pd
import numpy as np
import pymongo
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from string import punctuation
from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers.merge import concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [3]:
# open up a Mongo Connection and create a cursor
myclient = pymongo.MongoClient("mongodb://127.0.0.1:27017/")
curlydb = myclient['Naturaly_Curly_db']
curly_collection = curlydb['curly_profiles']

In [4]:
# Access all the unique items and store them to a list
query_1 = curly_collection.find({})
test = []
for x in query_1:
    test.append(x['signature'])

# This is the number of total entire in the database
print(len(test)) 

532489


In [11]:
# Create a set of unique items in the db and inspect it
unique_sigs = list(set(test))
print(type(unique_sigs))
print(len(unique_sigs))
print(unique_sigs[5000:5003])

<class 'list'>
7961
["3b curls, big chop Nov 07\n\nCurrent fave products: Aubrey Organics Honeysuckle Rose Shampoo and Conditioner, Aveda Curl Enhancer, Aveda Flaxseed Aloe gel\n\nStill like: Ouidad CC gel\nDon't like: Deva ArcAngell", '2b waves\nMedium texture, high porosity\nBronde & shoulder length, trying to grow it past my shoulder blades.\nWent natural on June 4th, 2010\nMy hair is hungry for PROTEIN!! :munky2:', '-3A/Medium/Low porosity\n\n-Cg since 01/2012\n-Growing out since 11/2011\n\n-Co-wash: Suave Naturals']


In [19]:
# test a cleaning function on a small sample size
samples = unique_sigs[0:10]
samples

['',
 'Lions, Tigers and My Blog',
 "Very modified CG. Activate and B2B pomegranate Condish. Re:coil, Joiwhip, LOOB, HE set me up and orange bottle. Boots, cant' remember anymore.\nIn a place where dry and burned straight hair is so common my wild spanish curls are my greatest weapon!",
 'Mod CG since 06/08/2011-trying to figure it all out!\n\nHair type: 2b/2c, medium texture, high (??) porosity\nCo-Wash: Tresemme Naturals\nRO: AOHR\nLI: Tresemme Naturals\nStylers: Jessicurl RR + CC, AG Re:Coil (still trying to love this), and Curl Keeper\nGels: EcoStyler, BRHG, Got2B Spiked Up\nTried: Aloe (ok for frizz), FSG (so wanted this to work)',
 '3a/b.',
 "\nFine, curly/wavy, prone to breakage. Still figuring out curl pattern...3A on some days (only w/ product), 2C on others (but not coarse). Would love help on hair-typing! Low porosity (I think). I thought it was low-density, but a hairstylist told me I have tiny hairs growing in underneath that break, so that's why it's thin). Product junkie

In [20]:
lower_sample=[]
for sample in samples:
    lower_sample.append(sample.lower().split())
lower_sample


[[],
 ['lions,', 'tigers', 'and', 'my', 'blog'],
 ['very',
  'modified',
  'cg.',
  'activate',
  'and',
  'b2b',
  'pomegranate',
  'condish.',
  're:coil,',
  'joiwhip,',
  'loob,',
  'he',
  'set',
  'me',
  'up',
  'and',
  'orange',
  'bottle.',
  'boots,',
  "cant'",
  'remember',
  'anymore.',
  'in',
  'a',
  'place',
  'where',
  'dry',
  'and',
  'burned',
  'straight',
  'hair',
  'is',
  'so',
  'common',
  'my',
  'wild',
  'spanish',
  'curls',
  'are',
  'my',
  'greatest',
  'weapon!'],
 ['mod',
  'cg',
  'since',
  '06/08/2011-trying',
  'to',
  'figure',
  'it',
  'all',
  'out!',
  'hair',
  'type:',
  '2b/2c,',
  'medium',
  'texture,',
  'high',
  '(??)',
  'porosity',
  'co-wash:',
  'tresemme',
  'naturals',
  'ro:',
  'aohr',
  'li:',
  'tresemme',
  'naturals',
  'stylers:',
  'jessicurl',
  'rr',
  '+',
  'cc,',
  'ag',
  're:coil',
  '(still',
  'trying',
  'to',
  'love',
  'this),',
  'and',
  'curl',
  'keeper',
  'gels:',
  'ecostyler,',
  'brhg,',
  'got

In [29]:
tokens = word_tokenize(samples[2])
tokens

['Very',
 'modified',
 'CG',
 '.',
 'Activate',
 'and',
 'B2B',
 'pomegranate',
 'Condish',
 '.',
 'Re',
 ':',
 'coil',
 ',',
 'Joiwhip',
 ',',
 'LOOB',
 ',',
 'HE',
 'set',
 'me',
 'up',
 'and',
 'orange',
 'bottle',
 '.',
 'Boots',
 ',',
 'cant',
 "'",
 'remember',
 'anymore',
 '.',
 'In',
 'a',
 'place',
 'where',
 'dry',
 'and',
 'burned',
 'straight',
 'hair',
 'is',
 'so',
 'common',
 'my',
 'wild',
 'spanish',
 'curls',
 'are',
 'my',
 'greatest',
 'weapon',
 '!']

In [34]:
tokenizer = Tokenizer()